In [54]:
import glob
import pandas as pd
import numpy as np
import cv2
import json
from skimage.draw import polygon
from matplotlib import pyplot as plt
DEFAULT_PATH = "E:/few_data/"
PATH_TO_CSV_DATA = DEFAULT_PATH + "few_data/*.csv"
PATH_TO_IMAGES_DATA = DEFAULT_PATH + "few_data/*.jpg"

csvs = glob.glob(PATH_TO_CSV_DATA)
orig_img = cv2.imread(DEFAULT_PATH + "few_data/20210712_141048_857A_ACCC8EAF31F3_0.jpg")

all_df = []
for csv in csvs:
    cdf = pd.read_csv(csv)
    cdf['filename'] = cdf['filename']
    all_df.append(cdf)
df = pd.concat(all_df)
df = df.reset_index()


In [55]:
df

,index,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,0,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,0,"{""name"":""polygon"",""all_points_x"":[330,326,342,...",{}
1,1,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,1,"{""name"":""polygon"",""all_points_x"":[566,545,559,...",{}
2,2,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,2,"{""name"":""polygon"",""all_points_x"":[703,706,725,...",{}
3,3,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,3,"{""name"":""polygon"",""all_points_x"":[684,686,698,...",{}
4,4,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,4,"{""name"":""polygon"",""all_points_x"":[750,740,739,...",{}
...,...,...,...,...,...,...,...,...
1693,1693,20210712_144334_EB33_ACCC8EAF31F3_1620.jpg,354526,{},0,0,{},{}
1694,1694,20210712_144334_EB33_ACCC8EAF31F3_1650.jpg,362200,{},0,0,{},{}
1695,1695,20210712_144334_EB33_ACCC8EAF31F3_1680.jpg,345024,{},0,0,{},{}
1696,1696,20210712_144334_EB33_ACCC8EAF31F3_1710.jpg,348453,{},0,0,{},{}


In [56]:
df['region_shape_attributes'] = df['region_shape_attributes'].apply(json.loads)

In [57]:
# Remove unlabeled data
df = df[(df['region_count']>0)]
df

,index,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,0,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,0,"{'name': 'polygon', 'all_points_x': [330, 326,...",{}
1,1,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,1,"{'name': 'polygon', 'all_points_x': [566, 545,...",{}
2,2,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,2,"{'name': 'polygon', 'all_points_x': [703, 706,...",{}
3,3,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,3,"{'name': 'polygon', 'all_points_x': [684, 686,...",{}
4,4,20210712_141048_857A_ACCC8EAF31F3_0.jpg,497805,{},25,4,"{'name': 'polygon', 'all_points_x': [750, 740,...",{}
...,...,...,...,...,...,...,...,...
470,470,20210712_141048_857A_ACCC8EAF31F3_570.jpg,341393,{},22,17,"{'name': 'polygon', 'all_points_x': [1101, 111...",{}
471,471,20210712_141048_857A_ACCC8EAF31F3_570.jpg,341393,{},22,18,"{'name': 'polygon', 'all_points_x': [1141, 114...",{}
472,472,20210712_141048_857A_ACCC8EAF31F3_570.jpg,341393,{},22,19,"{'name': 'polygon', 'all_points_x': [1139, 113...",{}
473,473,20210712_141048_857A_ACCC8EAF31F3_570.jpg,341393,{},22,20,"{'name': 'polygon', 'all_points_x': [871, 897,...",{}


### Semantic segmentation

In [58]:
areas = df.groupby(['filename'])['region_shape_attributes'].apply(list)
names = df.groupby(['filename'])['filename'].apply(list)
indexes = df.groupby(['filename'])['index'].apply(list)
res = [item[0] for item in names]
areas

filename
20210712_141048_857A_ACCC8EAF31F3_0.jpg      [{'name': 'polygon', 'all_points_x': [330, 326...
20210712_141048_857A_ACCC8EAF31F3_120.jpg    [{'name': 'polygon', 'all_points_x': [312, 297...
20210712_141048_857A_ACCC8EAF31F3_150.jpg    [{'name': 'polygon', 'all_points_x': [831, 831...
20210712_141048_857A_ACCC8EAF31F3_180.jpg    [{'name': 'polygon', 'all_points_x': [999, 987...
20210712_141048_857A_ACCC8EAF31F3_210.jpg    [{'name': 'polygon', 'all_points_x': [447, 514...
20210712_141048_857A_ACCC8EAF31F3_240.jpg    [{'name': 'polygon', 'all_points_x': [319, 313...
20210712_141048_857A_ACCC8EAF31F3_270.jpg    [{'name': 'polygon', 'all_points_x': [248, 241...
20210712_141048_857A_ACCC8EAF31F3_30.jpg     [{'name': 'polygon', 'all_points_x': [335, 321...
20210712_141048_857A_ACCC8EAF31F3_300.jpg    [{'name': 'polygon', 'all_points_x': [622, 616...
20210712_141048_857A_ACCC8EAF31F3_330.jpg    [{'name': 'polygon', 'all_points_x': [443, 426...
20210712_141048_857A_ACCC8EAF31F3_360.jpg

In [59]:
def save_mask(is_instance=False, is_find_min_left=False):
    color = 255
    norm = 1
    lefts = []
    for img_idx in range(0, len(areas)): # specific image
        image_name = res[img_idx]
        index_list = indexes[img_idx]
        mask = np.zeros((orig_img.shape[0], orig_img.shape[1]), dtype=np.uint8)
        if is_instance:
            norm = int(255 / len(index_list))
        i = 1
        for index in index_list:
            poly = df.iloc[index]['region_shape_attributes']
            rr, cc = polygon(poly['all_points_y'], poly['all_points_x'])
            rr[rr > mask.shape[0] - 1] = mask.shape[0] - 1 # sometimes out of bounds
            cc[cc > mask.shape[1] - 1] = mask.shape[1] - 1
            if is_instance:
                color = i * norm
            mask[rr, cc] = color
            i += 1
            #plt.imshow(mask)
        if is_instance:
            cv2.imwrite(DEFAULT_PATH + 'few_data/mask_instance/'+ image_name, mask)
        else:
            cv2.imwrite(DEFAULT_PATH + 'few_data/mask/'+ image_name, mask)
        if is_find_min_left:
            positions = np.nonzero(mask)
            left = positions[1].min()
            lefts.append(left)
    return lefts

In [60]:
cutter = save_mask(is_instance=False, is_find_min_left=True)

In [81]:
def cut_images(cut_list, is_instance=False):
    cut_list = sorted(cut_list)
    min_val = cut_list[0] - 7 if cut_list[0] > 7 else cut_list[0]
    if is_instance:
        mask_read_path = DEFAULT_PATH + 'few_data/mask_instance/'
        mask_write_path = DEFAULT_PATH + 'few_data/mask_cut_instance/'
    else:
        mask_read_path = DEFAULT_PATH + 'few_data/mask/'
        mask_write_path = DEFAULT_PATH + 'few_data/mask_cut/'

    for img_idx in range(0, len(areas)):
        image_name = res[img_idx]
        cut_img = cv2.imread(DEFAULT_PATH +'few_data/' + image_name)[:, min_val:]
        cut_mask = cv2.imread(mask_read_path + image_name)[:, min_val:]
        cv2.imwrite(DEFAULT_PATH + 'few_data/img_cut/' + image_name, cut_img)
        cv2.imwrite(mask_write_path + image_name, cut_mask)

In [82]:
cut_images(cutter)

E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_0.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_120.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_150.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_180.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_210.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_240.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_270.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_30.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_300.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_330.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_360.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_390.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_420.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_450.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_480.jpg
E:/few_data/few_data/20210712_141048_857A_ACCC8EAF31F3_510

### Instance segmentation

In [ ]:
save_mask(is_instance=True)